In [4]:
import numpy as np

# -----------------
# Regression Losses
# -----------------
def mse(y, y_pred):
    y, y_pred = np.asarray(y), np.asarray(y_pred)
    return np.mean((y - y_pred) ** 2)

def rmse(y, y_pred):
    return np.sqrt(mse(y, y_pred))

def mae(y, y_pred):
    y, y_pred = np.asarray(y), np.asarray(y_pred)
    return np.mean(np.abs(y - y_pred))

def huber(y, y_pred, delta=1.0):
    y, y_pred = np.asarray(y), np.asarray(y_pred)
    r = y - y_pred
    mask = np.abs(r) <= delta
    return np.mean(0.5 * r[mask]**2 + delta*(np.abs(r[~mask]) - 0.5*delta))

def log_cosh(y, y_pred):
    y, y_pred = np.asarray(y), np.asarray(y_pred)
    return np.mean(np.log(np.cosh(y_pred - y)))

def quantile_loss(y, y_pred, q=0.9):
    """Pinball/Quantile loss: q in (0,1)."""
    y, y_pred = np.asarray(y), np.asarray(y_pred)
    e = y - y_pred
    return np.mean(np.maximum(q*e, (q-1)*e))

# --------------------
# Classification Losses
# --------------------
def binary_cross_entropy(y, p, eps=1e-12):
    """y in {0,1}; p is predicted probability of class 1."""
    y, p = np.asarray(y), np.clip(np.asarray(p), eps, 1-eps)
    return np.mean(-(y*np.log(p) + (1-y)*np.log(1-p)))

def categorical_cross_entropy(y_onehot, p, eps=1e-12):
    """y_onehot: (n,k) one-hot targets; p: (n,k) predicted probs."""
    y_onehot, p = np.asarray(y_onehot), np.clip(np.asarray(p), eps, 1-eps)
    return -np.mean(np.sum(y_onehot * np.log(p), axis=1))

def hinge(y, y_score):
    """y in {-1,+1}; y_score is raw signed score."""
    y, y_score = np.asarray(y), np.asarray(y_score)
    return np.mean(np.maximum(0.0, 1 - y*y_score))

def squared_hinge(y, y_score):
    y, y_score = np.asarray(y), np.asarray(y_score)
    return np.mean(np.maximum(0.0, 1 - y*y_score) ** 2)

def focal_loss_binary(y, p, gamma=2.0, alpha=0.25, eps=1e-12):
    """Binary focal loss used for class imbalance."""
    y, p = np.asarray(y), np.clip(np.asarray(p), eps, 1-eps)
    pt = np.where(y == 1, p, 1-p)
    alpha_t = np.where(y == 1, alpha, 1-alpha)
    return np.mean(-alpha_t * (1-pt)**gamma * np.log(pt))


In [5]:
# Regression demo
rng = np.random.default_rng(0)
y = rng.normal(0, 1, 10)
y_pred = y + rng.normal(0, 0.5, 10)

print("MSE:", mse(y, y_pred))
print("MAE:", mae(y, y_pred))
print("Huber:", huber(y, y_pred, delta=1.0))
print("LogCosh:", log_cosh(y, y_pred))
print("Quantile(0.9):", quantile_loss(y, y_pred, q=0.9))

# Classification demo (binary)
y_bin = rng.integers(0, 2, 12)
p_bin = np.clip(rng.uniform(0, 1, 12), 1e-6, 1-1e-6)
print("BCE:", binary_cross_entropy(y_bin, p_bin))
print("Focal:", focal_loss_binary(y_bin, p_bin, gamma=2, alpha=0.25))

# Hinge needs labels in {-1, +1} and raw scores
y_pm = np.where(y_bin==1, 1, -1)
scores = rng.normal(0, 1, 12)
print("Hinge:", hinge(y_pm, scores))
print("Squared Hinge:", squared_hinge(y_pm, scores))

# Multiclass cross-entropy
n, k = 5, 3
y_idx = rng.integers(0, k, n)
y_onehot = np.eye(k)[y_idx]
logits = rng.normal(size=(n, k))
# softmax for probs
p = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)
print("Categorical CE:", categorical_cross_entropy(y_onehot, p))


MSE: 0.2396215951994023
MAE: 0.3750652113727884
Huber: 0.7205583832216177
LogCosh: 0.10650460025778982
Quantile(0.9): 0.27773989482894745
BCE: 1.7769408820901214
Focal: 0.8760853689396217
Hinge: 1.3482071704651235
Squared Hinge: 2.9116271759001258
Categorical CE: 1.4788490256188567


In [6]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import log_loss, hinge_loss, accuracy_score
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=1500, n_features=20, random_state=0)
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.25, random_state=0)

# Probabilistic model
logreg = LogisticRegression(max_iter=500).fit(Xtr, ytr)
p = logreg.predict_proba(Xte)
print("Log Loss (BCE):", log_loss(yte, p))

# Large-margin model
sgd_svm = SGDClassifier(loss="hinge", random_state=0).fit(Xtr, ytr)
margin = sgd_svm.decision_function(Xte)
print("Hinge Loss:", hinge_loss(yte*2-1, margin))  # convert to {-1,+1}
print("Accuracy:", accuracy_score(yte, sgd_svm.predict(Xte)))


Log Loss (BCE): 0.24481385579506365
Hinge Loss: 0.42566816628214665
Accuracy: 0.8853333333333333
